# Sample Size 

In [1]:
# calculators
# http://www.evanmiller.org/ab-testing/sample-size.html
# https://www.optimizely.com/sample-size-calculator/
# https://docs.adobe.com/content/target-microsite/testcalculator.html

In [2]:
from math import *
from scipy.stats import norm as norm

In [3]:
def ppnd(p):
    a0 = 2.50662823884
    a1 = -18.61500062529
    a2 = 41.39119773534
    a3 = -25.44106049637
    b1 = -8.47351093090
    b2 = 23.08336743743
    b3 = -21.06224101826
    b4 = 3.13082909833
    c0 = -2.78718931138
    c1 = -2.29796479134
    c2 = 4.85014127135
    c3 = 2.32121276858
    d1 = 3.54388924762
    d2 = 1.63706781897
    r = 0
    split = 0.42
    value =0

    # 0.08 < P < 0.92
    if abs(p - 0.5) <= split:
        r = (p - 0.5) * (p - 0.5)

        value = (p - 0.5) * (((
                    a3 * r +
                    a2) * r +
                a1) * r +
            a0) / ((((
                        b4 * r +
                        b3) * r +
                    b2) * r +
                b1) * r +
            1.0)
    
    # P < 0.08 or P > 0.92
    # R = min ( P, 1-P )
    elif 0.0 < p and p < 1.0:
        if 0.5 < p:
            r = sqrt(-log(1.0 - p))
        else:
            r = sqrt(-log(p))
        

        value = (((
                    c3 * r +
                    c2) * r +
                c1) * r +
            c0) / ((
                d2 * r +
                d1) * r +
            1.0)

        if p < 0.5:
            value = -value
        
    
    # P <= 0.0 or 1.0 <= P
    else:
        value = None
    

    return value

In [4]:
def num_subjects(alpha, power_level, p, delta):
    if p > 0.5:
        p = 1 - p
    
    alpha2 = ppnd(1-alpha/2)
    beta = ppnd(power_level)
    
    sd1 = sqrt(2 * p * (1 - p))
    sd2 = sqrt(p * (1 - p) + (p + delta) * (1 - p - delta))
    
    return (alpha2 * sd1 + beta * sd2) ** 2 / delta ** 2

In [5]:
# rewrite remove ppnd
def num_subjects_2(alpha, power_level, p, delta):
    if p > 0.5:
        p = 1 - p
    
    alpha2 =  -1 * norm.isf([1-alpha/2])[0] # ppnd(1-alpha/2)
    beta = -1 * norm.isf([power_level])[0] # ppnd(power_level)
    
    sd1 = sqrt(2 * p * (1 - p))
    sd2 = sqrt(p * (1 - p) + (p + delta) * (1 - p - delta))
    
    return (alpha2 * sd1 + beta * sd2) ** 2 / delta ** 2

In [6]:
def sample_power_probtest(p1, p2, power=0.8, sig=0.05):
    z = norm.isf([sig/2]) #two-sided t test
    zp = -1 * norm.isf([power]) 
    d = (p1-p2)
    s =2*((p1+p2) /2)*(1-((p1+p2) /2))
    n = s * ((zp + z)**2) / (d**2)
    #return int(round(n[0]))
    #return int(n[0])
    return n[0]

In [7]:
sign = 0.05
power = 0.8
base_conv = 0.1
detectable_effect_rel = 0.1

print(num_subjects(sign, power, base_conv, detectable_effect_rel*base_conv))
print(num_subjects_2(sign, power, base_conv, detectable_effect_rel*base_conv))
print(sample_power_probtest(base_conv, base_conv*(1+detectable_effect_rel), power, sign))

14312.856232802844
14312.8562419
14751.9694607
